In [11]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns
import re
from scipy import stats
from tqdm.notebook import tqdm
import time 
from scipy.stats import zscore
from scipy.stats import f_oneway
import statsmodels.stats.anova as anova
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.anova import AnovaRM
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import statsmodels.api as sm
import statsmodels.formula.api as smf


# Behavioural Data Analysis

In [12]:
#dataset_path = r"C:\Users\Nabiya\OneDrive - Duke University\Research\DTI x MetaEmotion\Paper_Drafting\dataset"
dataset_path = r"dataset"

dataframes = []

for filename in os.listdir(dataset_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(dataset_path, filename)
        df = pd.read_csv(file_path)
        dataframes.append(df)
all_participants_dfs = pd.concat(dataframes, ignore_index=True)

In [13]:
# cleaning the data 
all_participants_dfs = all_participants_dfs.loc[all_participants_dfs['block'] != 'prc']

all_participants_dfs = all_participants_dfs.groupby('ID').apply(lambda x: x[np.abs(x['Decision RT'] - x['Decision RT'].mean()) <= (3 * x['Decision RT'].std())])


C:\Users\Nabiya\AppData\Local\Temp\ipykernel_14864\1194084739.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  all_participants_dfs = all_participants_dfs.groupby('ID').apply(lambda x: x[np.abs(x['Decision RT'] - x['Decision RT'].mean()) <= (3 * x['Decision RT'].std())])


In [14]:
all_participants_dfs['Accuracy'] = (all_participants_dfs['response'] == all_participants_dfs['Ans']).astype(int)
all_participants_dfs.columns

Index(['instr_resp.rt', 'response', 'accuracy', 'Decision RT', 'Confidence',
       'Confidence RT', 'rest rt', 'prac_loop.thisRepN',
       'prac_loop.thisTrialN', 'prac_loop.thisN', 'prac_loop.thisIndex',
       'prac_loop.ran', 'block', 'condition', 'face', 'choice1', 'choice2',
       'emotion', 'intensity', 'Ans', 'ID', '姓名', '年龄', '性别', 'date',
       'expName', 'psychopyVersion', 'OS', 'frameRate', 'block1.thisRepN',
       'block1.thisTrialN', 'block1.thisN', 'block1.thisIndex', 'block1.ran',
       'block2.thisRepN', 'block2.thisTrialN', 'block2.thisN',
       'block2.thisIndex', 'block2.ran', 'Accuracy'],
      dtype='object')

In [15]:
all_participants_dfs.reset_index(drop=True, inplace=True)

In [16]:
print(all_participants_dfs['ID'].unique())
print(all_participants_dfs['姓名'].unique())


[ 1  3  4  5  6  7  8  9 10 11 12 13 14 17 18 19 20 21 22 23 24 25 27 28
 31 32 34 36 39]
['王少刚' '王辉' '董正心' '高倩' '叶帅' '李闯' '钱文莉' '李诗晨' '王梦宣' '姚林林' '赵冰冰' '李思璇' '何金辉'
 '吴建星' 'bq' 'gxy' 'CTG' 'zwl' nan 'MZY' '林增萍' 'qdl' '庄惠翔' '焦雄' '郭有容' '余鹏程'
 '陈若羽' 'Xuewei Qin']


In [17]:
# group by emotion and take the mean Confidence, Decision RT, and Accuracy. 
# I want a separate dataframe for each of these variables, aka one df for Confidence, one for Decision RT, and one for Accuracy.

# important
#######Confidence
confidence_df = all_participants_dfs.groupby(['ID', '姓名', 'emotion'])['Confidence'].mean().reset_index()

######Decision RT
decision_rt_df = all_participants_dfs.groupby(['ID', '姓名', 'emotion'])['Decision RT'].mean().reset_index()

#####Accuracy
accuracy_df = all_participants_dfs.groupby(['ID', '姓名', 'emotion'])['Accuracy'].mean().reset_index()

# rename 姓名 to Name and emotion to Emotion in confidence_df, decision_rt_df, and accuracy_df
confidence_df = confidence_df.rename(columns={'姓名': 'Name', 'emotion': 'Emotion', 'Confidence': 'Confidence'})
decision_rt_df = decision_rt_df.rename(columns={'姓名': 'Name', 'emotion': 'Emotion', 'Decision RT': 'Decision_RT'})
accuracy_df = accuracy_df.rename(columns={'姓名': 'Name', 'emotion': 'Emotion', 'Accuracy': 'Accuracy'})

# change order of columns to Name, ID, emotion, Confidence, Decision RT, Accuracy
confidence_df = confidence_df[['Name', 'ID', 'Emotion', 'Confidence']]
decision_rt_df = decision_rt_df[['Name', 'ID', 'Emotion', 'Decision_RT']]
accuracy_df = accuracy_df[['Name', 'ID', 'Emotion', 'Accuracy']]

# print heads
print(confidence_df.head())
print(decision_rt_df.head())
print(accuracy_df.head())

# merge or join the three dataframes on Name, ID, and emotion

  Name  ID  Emotion  Confidence
0  王少刚   1  Disgust    3.170213
1  王少刚   1     Fear    3.104167
2  王少刚   1    Happy    3.625000
3   王辉   3  Disgust    3.354167
4   王辉   3     Fear    3.422222
  Name  ID  Emotion  Decision_RT
0  王少刚   1  Disgust     1.274849
1  王少刚   1     Fear     1.334447
2  王少刚   1    Happy     0.870779
3   王辉   3  Disgust     1.770416
4   王辉   3     Fear     1.501141
  Name  ID  Emotion  Accuracy
0  王少刚   1  Disgust  0.617021
1  王少刚   1     Fear  0.500000
2  王少刚   1    Happy  0.812500
3   王辉   3  Disgust  0.812500
4   王辉   3     Fear  0.577778


In [18]:
confidence_df.to_csv('All_partis_confidence.csv', index=False)
decision_rt_df.to_csv('All_partis_decRT.csv', index=False)
accuracy_df.to_csv('All_partis_accuracy.csv', index=False)

In [19]:
mratio_df = pd.read_excel(r"All_partis_mratio.xlsx")
dprime_df = pd.read_excel(r"All_partis_dprime.xlsx")
logmratio_df = pd.read_excel(r"All_partis_logMratio.xlsx")
metadprime_df = pd.read_excel(r"All_partis_metad'.xlsx")
dti_df = pd.read_csv(r"mratio - DTI data.csv")

In [20]:
# print columns from each dataframe

for df in [confidence_df, decision_rt_df, accuracy_df, mratio_df, dprime_df, logmratio_df, metadprime_df, dti_df]:
    print(f"{df.columns}")

Index(['Name', 'ID', 'Emotion', 'Confidence'], dtype='object')
Index(['Name', 'ID', 'Emotion', 'Decision_RT'], dtype='object')
Index(['Name', 'ID', 'Emotion', 'Accuracy'], dtype='object')
Index(['Name', 'ID', 'All_emotions', 'Disgust', 'Happy', 'Fear'], dtype='object')
Index(['Name', 'ID', 'All_emotions', 'Disgust', 'Happy', 'Fear'], dtype='object')
Index(['Name', 'ID', 'All_emotions', 'Disgust', 'Happy', 'Fear'], dtype='object')
Index(['Name', 'ID', 'All_emotions', 'Disgust', 'Happy', 'Fear'], dtype='object')
Index(['ID', '年龄', '性别', '采集日期', 'FA_Fornix', 'FA_Cingulum_L', 'FA_Cingulum_R',
       'FA_SLF_L', 'FA_SLF_R', 'MD_Fornix', 'MD_Cingulum_L', 'MD_Cingulum_R',
       'MD_SLF_L', 'MD_SLF_R', 'AD_Fornix', 'AD_Cingulum_L', 'AD_Cingulum_R',
       'AD_SLF_L', 'AD_SLF_R', 'RD_Fornix', 'RD_Cingulum_L', 'RD_Cingulum_R',
       'RD_SLF_L', 'RD_SLF_R', 'FA_Cingulum (hippocampus)_L',
       'FA_Cingulum (hippocampus)_R', 'FA_Fornix (cres)_L',
       'FA_Fornix (cres)_R', 'MD_Cingulum (hippo

In [21]:
# remove 'All_emotions' column from mratio_df, dprime_df, logmratio_df, metadprime_df
mratio_df = mratio_df.drop(columns=['All_emotions'])
dprime_df = dprime_df.drop(columns=['All_emotions'])
logmratio_df = logmratio_df.drop(columns=['All_emotions'])
metadprime_df = metadprime_df.drop(columns=['All_emotions'])

# melt mratio_df, dprime_df, logmratio_df, and meta_dprime_df
mratio_df = pd.melt(mratio_df, id_vars=['Name', 'ID'], var_name='Emotion', value_name='MRatio')
dprime_df = pd.melt(dprime_df, id_vars=['Name', 'ID'], var_name='Emotion', value_name='dprime')
logmratio_df = pd.melt(logmratio_df, id_vars=['Name', 'ID'], var_name='Emotion', value_name='logMRatio')
metadprime_df = pd.melt(metadprime_df, id_vars=['Name', 'ID'], var_name='Emotion', value_name='metadprime')

# sample heads 
mratio_df.head()
dprime_df.head()
accuracy_df.head()
decision_rt_df.head()


,Name,ID,Emotion,Decision_RT
0,王少刚,1,Disgust,1.274849
1,王少刚,1,Fear,1.334447
2,王少刚,1,Happy,0.870779
3,王辉,3,Disgust,1.770416
4,王辉,3,Fear,1.501141


In [22]:
confidence_df.Confidence.describe()

count    81.000000
mean      3.482143
std       0.262184
min       2.644444
25%       3.340426
50%       3.520833
75%       3.659091
max       3.916667
Name: Confidence, dtype: float64

In [23]:
dfs = [accuracy_df, decision_rt_df, confidence_df, mratio_df, dprime_df, logmratio_df, metadprime_df]
# print columsn 
for df in dfs:
    print(df.columns)

Index(['Name', 'ID', 'Emotion', 'Accuracy'], dtype='object')
Index(['Name', 'ID', 'Emotion', 'Decision_RT'], dtype='object')
Index(['Name', 'ID', 'Emotion', 'Confidence'], dtype='object')
Index(['Name', 'ID', 'Emotion', 'MRatio'], dtype='object')
Index(['Name', 'ID', 'Emotion', 'dprime'], dtype='object')
Index(['Name', 'ID', 'Emotion', 'logMRatio'], dtype='object')
Index(['Name', 'ID', 'Emotion', 'metadprime'], dtype='object')


In [24]:
# stats_results path 
results_path = r"Main Results"

In [25]:
#Conduct anova with repeated measures for each of the dataframes 

dfs = [accuracy_df, decision_rt_df, confidence_df, mratio_df, dprime_df, logmratio_df, metadprime_df]
df_names = ['Accuracy', 'Decision_RT', 'Confidence', 'MRatio', 'dprime', 'logMRatio', 'metadprime']
anova_results = {}
i = 0  
for df in dfs:
    dependent_variable = df.columns[-1]
    aovrm = AnovaRM(df, depvar=dependent_variable, subject='ID', within=['Emotion'])
    res = aovrm.fit()
    anova_results[df_names[i]] = res
    i += 1


# Do post-hoc tests using Tukey's HSD for each of the dataframes

tukey_results = {}
i = 0
for df in dfs:
    dependent_variable = df.columns[-1]
    # check if the anova result is significant
    with open(os.path.join(results_path, f'{df_names[i]}_anova_results.txt'), 'w') as f:
        f.write(f"ANOVA Results for {df_names[i]}\n\n")
        f.write(str(anova_results[df_names[i]]))
        pvalue = anova_results['Accuracy'].summary().tables[0].loc['Emotion', 'Pr > F']
        if pvalue < 0.05:
            tukey = pairwise_tukeyhsd(endog=df[dependent_variable], groups=df['Emotion'], alpha=0.05)
            tukey_results[df_names[i]] = tukey
            # save the tukey results to the text file opened above
            f.write("\n\n\n")
            f.write(f"Tukey Results for {df_names[i]}\n\n")
            f.write(str(tukey))
        else:
            f.write("\n\n\n")
            f.write("No significant results")
    i += 1

In [ ]:
# we now have all dfs, all anova results, and all post hoc results. we shall plot. 

# produce a box plot for each dataframe, with the x axis as Emotion and the y axis as the dependent variable... 
# in the occurence of a significant anova result, we can add the post hoc results to the plot. 
# import plotly

import plotly.express as px

dfs = [accuracy_df, decision_rt_df, confidence_df, mratio_df, dprime_df, logmratio_df, metadprime_df]
df_names = ['Accuracy', 'Decision_RT', 'Confidence', 'MRatio', 'dprime', 'logMRatio', 'metadprime']

for df in dfs:
    dependent_variable = df.columns[-1]
    # use plotly 
    fig = px.box(df, x='Emotion', y=dependent_variable, points='all', title=f'{dependent_variable} by Emotion', 
                    hover_data=['Name', 'ID', 'Emotion', dependent_variable], color='Emotion')
    fig.update_layout(yaxis_title=dependent_variable, width=1200, height=600)
    # check if the anova result is significant
    pvalue = anova_results[dependent_variable].summary().tables[0].loc['Emotion', 'Pr > F']
    if pvalue < 0.05:
        tukey = tukey_results[dependent_variable]
        # print anova and tukey results
        print(f"ANOVA Results for {dependent_variable}")
        print(anova_results[dependent_variable].summary())
        print(tukey.summary())
        # add pvalue to the plot
        fig.add_annotation(text=f"p-value: {pvalue:.2f} (Anova Repeated Measures)", x=0.9, 
                           y = 1.2, 
                            showarrow=False, 
                            font=dict(
                                family="Courier New, monospace",
                                size=16,
                                color="#ffffff"
                            ), xref="paper", yref="paper", bgcolor="#1f77b4", bordercolor="#1f77b4", borderwidth=2, borderpad=4, opacity=0.8)
    else:
        fig.add_annotation(text="No significant difference", x=0.9,
                            y = 1.1, 
                             showarrow=False, 
                             font=dict(
                                  family="Courier New, monospace",
                                  size=16,
                                  color="#ffffff"
                             ), xref="paper", yref="paper", bgcolor="#1f77b4", bordercolor="#1f77b4", borderwidth=2, borderpad=4, opacity=0.8)

    fig.show()


ANOVA Results for Accuracy
                Anova
        F Value Num DF  Den DF Pr > F
-------------------------------------
Emotion 90.4813 2.0000 52.0000 0.0000

 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
 group1 group2 meandiff p-adj   lower   upper  reject
-----------------------------------------------------
Disgust   Fear  -0.1724    0.0 -0.2238 -0.1211   True
Disgust  Happy   0.0844 0.0005   0.033  0.1357   True
   Fear  Happy   0.2568    0.0  0.2054  0.3082   True
-----------------------------------------------------


ANOVA Results for Decision_RT
                Anova
        F Value Num DF  Den DF Pr > F
-------------------------------------
Emotion 62.1044 2.0000 52.0000 0.0000

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
 group1 group2 meandiff p-adj  lower   upper  reject
----------------------------------------------------
Disgust   Fear   0.0534 0.912 -0.2585  0.3654  False
Disgust  Happy  -0.7008   0.0 -1.0128 -0.3889   True
   Fear  Happy  -0.7542   0.0 -1.0662 -0.4423   True
----------------------------------------------------


ANOVA Results for Confidence
                Anova
        F Value Num DF  Den DF Pr > F
-------------------------------------
Emotion 37.9244 2.0000 52.0000 0.0000

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
 group1 group2 meandiff p-adj   lower  upper  reject
----------------------------------------------------
Disgust   Fear  -0.0587 0.6047 -0.2049 0.0875  False
Disgust  Happy   0.2603 0.0002  0.1141 0.4065   True
   Fear  Happy    0.319    0.0  0.1728 0.4652   True
----------------------------------------------------


ANOVA Results for dprime
                Anova
        F Value Num DF  Den DF Pr > F
-------------------------------------
Emotion 12.7007 2.0000 56.0000 0.0000

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
 group1 group2 meandiff p-adj   lower  upper  reject
----------------------------------------------------
Disgust   Fear  -0.1093 0.6773 -0.4187    0.2  False
Disgust  Happy   0.4789 0.0011  0.1695 0.7882   True
   Fear  Happy   0.5882 0.0001  0.2788 0.8976   True
----------------------------------------------------


ANOVA Results for metadprime
                Anova
        F Value Num DF  Den DF Pr > F
-------------------------------------
Emotion 37.3750 2.0000 56.0000 0.0000

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
 group1 group2 meandiff p-adj   lower  upper  reject
----------------------------------------------------
Disgust   Fear  -0.0971 0.8895 -0.5991  0.405  False
Disgust  Happy   1.2761    0.0   0.774 1.7781   True
   Fear  Happy   1.3731    0.0  0.8711 1.8752   True
----------------------------------------------------


# Correlating with DTI Data (LogMRatio Only)

In [27]:
tracts = [column for column in dti_df.columns if ('Cingulum' in column or 'IFO' in column or 'SLF' in column or 'UF' in column or 'Genu' in column) and not any(substring in column for substring in ('AD', 'RD'))]
tractdict = {'IFO': ['FA_IFO_L', 'FA_IFO_R', 'MD_IFO_L', 'MD_IFO_R'],
             'SLF': ['FA_SLF_L', 'FA_SLF_R', 'MD_SLF_L', 'MD_SLF_R'],
             'UF': ['FA_UF_L', 'FA_UF_R', 'MD_UF_L', 'MD_UF_R'],
             'Cingulum': ['FA_Cingulum_L', 'FA_Cingulum_R', 'MD_Cingulum_L', 'MD_Cingulum_R'],
             'Cingulum (hippocampus)': ['FA_Cingulum (hippocampus)_L', 'FA_Cingulum (hippocampus)_R', 'MD_Cingulum (hippocampus)_L', 'MD_Cingulum (hippocampus)_R']}

# merge dti_df with logmratio_df 
logmratio_dti = pd.merge(logmratio_df, dti_df, on=['ID'], how='inner')
logmratio_dti.head()

,Name,ID,Emotion,logMRatio,年龄,性别,采集日期,FA_Fornix,FA_Cingulum_L,FA_Cingulum_R,...,MD_SFO_L,MD_SFO_R,AD_SFO_L,AD_SFO_R,RD_SFO_L,RD_SFO_R,FA_Genu,MD_Genu,AD_Genu,RD_Genu
0,王少刚,1,Disgust,-0.430963,30.0,男,10/17/20,0.573723,0.622870,0.557041,...,0.000488,0.000494,0.000841,0.000827,0.000312,0.000328,0.639504,0.000566,0.001053,0.000323
1,王梦宣,10,Disgust,0.391020,29.0,女,10/25/20,0.451164,0.606037,0.550508,...,0.000524,0.000518,0.000871,0.000852,0.000350,0.000350,0.612827,0.000608,0.001092,0.000366
2,姚林林,11,Disgust,0.179413,26.0,女,10/25/20,0.570839,0.604930,0.525909,...,0.000481,0.000500,0.000853,0.000877,0.000296,0.000311,0.642949,0.000566,0.001051,0.000323
3,赵冰冰,12,Disgust,0.140267,26.0,女,10/25/20,0.538577,0.655879,0.625258,...,0.000514,0.000505,0.000885,0.000852,0.000328,0.000331,0.656406,0.000569,0.001073,0.000317
4,李思璇,13,Disgust,0.434430,26.0,女,10/25/20,0.601769,0.604695,0.595042,...,0.000487,0.000490,0.000815,0.000830,0.000322,0.000320,0.639033,0.000573,0.001060,0.000329


In [28]:
logmratio_dti['性别']

Index(['Name', 'ID', 'Emotion', 'logMRatio', '年龄', '性别', '采集日期', 'FA_Fornix',
       'FA_Cingulum_L', 'FA_Cingulum_R', 'FA_SLF_L', 'FA_SLF_R', 'MD_Fornix',
       'MD_Cingulum_L', 'MD_Cingulum_R', 'MD_SLF_L', 'MD_SLF_R', 'AD_Fornix',
       'AD_Cingulum_L', 'AD_Cingulum_R', 'AD_SLF_L', 'AD_SLF_R', 'RD_Fornix',
       'RD_Cingulum_L', 'RD_Cingulum_R', 'RD_SLF_L', 'RD_SLF_R',
       'FA_Cingulum (hippocampus)_L', 'FA_Cingulum (hippocampus)_R',
       'FA_Fornix (cres)_L', 'FA_Fornix (cres)_R',
       'MD_Cingulum (hippocampus)_L', 'MD_Cingulum (hippocampus)_R',
       'MD_Fornix (cres)_L', 'MD_Fornix (cres)_R',
       'AD_Cingulum (hippocampus)_L', 'AD_Cingulum (hippocampus)_R',
       'AD_Fornix (cres)_L', 'AD_Fornix (cres)_R',
       'RD_Cingulum (hippocampus)_L', 'RD_Cingulum (hippocampus)_R',
       'RD_Fornix (cres)_L', 'RD_Fornix (cres)_R', 'FA_UF_L', 'FA_UF_R',
       'MD_UF_L', 'MD_UF_R', 'AD_UF_L', 'AD_UF_R', 'RD_UF_L', 'RD_UF_R',
       'FA_ACR_L', 'FA_ACR_R', 'MD_ACR_L', 'MD_A

In [31]:
dicti = get_gender()

In [32]:
dicti

{1: '男',
 10: '女',
 11: '女',
 12: '女',
 13: '女',
 14: '男',
 17: '男',
 18: '女',
 19: '女',
 20: '男',
 21: '男',
 22: '男',
 23: '女',
 24: '女',
 25: '女',
 27: '男',
 28: '男',
 3: '男',
 31: '女',
 32: '男',
 34: '女',
 36: '男',
 39: '女',
 4: '女',
 5: '女',
 6: '男',
 7: '男',
 8: '女',
 9: '女'}

### Ignore this block as it is not key to the analysis

In [16]:
# just for fun, let's throw a boxplot to see if there are any differences in the dti data between the patients 
# agnostic of the emotion.
# the x axis is all the columns with FA and MD in the name, and the is the value of the column.

from plotly import graph_objects as go

data_llll = []
for tract, columns in tractdict.items():
    for column in columns:
        for value in logmratio_dti[column]:
            data_llll.append({'Tract': tract, 'Column': column, 'Value': value})
df_llll = pd.DataFrame(data_llll)
# create new column to discern between FA and MD
df_llll['FA/MD'] = df_llll['Column'].apply(lambda x: x.split('_')[0])
# rename columns, from value to FA/MD value
df_llll = df_llll.rename(columns={'Value': 'FA/MD Value', 'Column': 'Tract/FA/MD'})
# separate into FA and MD dataframes
df_fa = df_llll[df_llll['FA/MD'] == 'FA']
df_md = df_llll[df_llll['FA/MD'] == 'MD']

# create boxplot for FA and MD data
fig = px.box(df_fa, x='Tract/FA/MD', y='FA/MD Value', points='all', title='FA Values by Tract', color='Tract', 
                hover_data=['Tract', 'Tract/FA/MD', 'FA/MD Value']).update_traces(
                marker=dict(size=4, opacity=0.6), jitter=0.3,
                boxmean=True, line=dict(width=4))
fig.update_layout(yaxis_title='FA Value', width=1200, height=600, showlegend=False)
fig.show()

fig = px.box(df_md, x='Tract/FA/MD', y='FA/MD Value', points='all', title='MD Values by Tract', color='Tract',
                hover_data=['Tract', 'Tract/FA/MD', 'FA/MD Value']).update_traces(
                marker=dict(size=4, opacity=0.6), jitter=0.3,
                boxmean=True, line=dict(width=4))
fig.update_layout(yaxis_title='MD Value', width=1200, height=600, showlegend=False)
fig.show()


In [17]:
# let's do some sanity checks to see if a single patient has multiple entries for the same emotion
# there should be 3 entries for each patient, one for each emotion.
# let's check if this is the case for all patients.
#  and let's check if there are any patients with less than 3 entries. and let's check if the entries are the same for each emotion. which they should be.
# We are dealing with the logmratio_dti dataframe.

# check if there are any patients with less than 3 entries
patients = logmratio_dti['ID'].unique()
for patient in patients:
    patient_df = logmratio_dti[logmratio_dti['ID'] == patient]
    if patient_df.shape[0] != 3:
        print(f"Patient {patient} has {patient_df.shape[0]} entries")
    
    # check if the entries are the same for some dti columns
    for column in ['FA_IFO_L', 'FA_IFO_R', 'MD_IFO_L', 'MD_IFO_R']:
        if patient_df[column].nunique() != 1:
            print(f"Patient {patient} has different values for {column}")

# if nothing is printed, this means we are good. 


In [18]:
logmratio_dti.head()

,Name,ID,Emotion,logMRatio,年龄,性别,采集日期,FA_Fornix,FA_Cingulum_L,FA_Cingulum_R,...,MD_SFO_L,MD_SFO_R,AD_SFO_L,AD_SFO_R,RD_SFO_L,RD_SFO_R,FA_Genu,MD_Genu,AD_Genu,RD_Genu
0,王少刚,1,Disgust,-0.430963,30.0,男,10/17/20,0.573723,0.622870,0.557041,...,0.000488,0.000494,0.000841,0.000827,0.000312,0.000328,0.639504,0.000566,0.001053,0.000323
1,王梦宣,10,Disgust,0.391020,29.0,女,10/25/20,0.451164,0.606037,0.550508,...,0.000524,0.000518,0.000871,0.000852,0.000350,0.000350,0.612827,0.000608,0.001092,0.000366
2,姚林林,11,Disgust,0.179413,26.0,女,10/25/20,0.570839,0.604930,0.525909,...,0.000481,0.000500,0.000853,0.000877,0.000296,0.000311,0.642949,0.000566,0.001051,0.000323
3,赵冰冰,12,Disgust,0.140267,26.0,女,10/25/20,0.538577,0.655879,0.625258,...,0.000514,0.000505,0.000885,0.000852,0.000328,0.000331,0.656406,0.000569,0.001073,0.000317
4,李思璇,13,Disgust,0.434430,26.0,女,10/25/20,0.601769,0.604695,0.595042,...,0.000487,0.000490,0.000815,0.000830,0.000322,0.000320,0.639033,0.000573,0.001060,0.000329


## Correlation block is here. However, we may add more tracts, and in that occasion, all you have to do is add them to tractdict

In [19]:
tracts = [column for column in dti_df.columns if ('Cingulum' in column or 'IFO' in column or 'SLF' in column or 'UF' in column or 'Genu' in column)]
tractdict = {
    'IFO': ['FA_IFO_L', 'FA_IFO_R', 'MD_IFO_L', 'MD_IFO_R', 'AD_IFO_L', 'AD_IFO_R', 'RD_IFO_L', 'RD_IFO_R'],
    'SLF': ['FA_SLF_L', 'FA_SLF_R', 'MD_SLF_L', 'MD_SLF_R', 'AD_SLF_L', 'AD_SLF_R', 'RD_SLF_L', 'RD_SLF_R'],
    'UF': ['FA_UF_L', 'FA_UF_R', 'MD_UF_L', 'MD_UF_R', 'AD_UF_L', 'AD_UF_R', 'RD_UF_L', 'RD_UF_R'],
    'Cingulum': ['FA_Cingulum_L', 'FA_Cingulum_R', 'MD_Cingulum_L', 'MD_Cingulum_R', 'AD_Cingulum_L', 'AD_Cingulum_R', 'RD_Cingulum_L', 'RD_Cingulum_R'],
    'Cingulum (hippocampus)': ['FA_Cingulum (hippocampus)_L', 'FA_Cingulum (hippocampus)_R', 'MD_Cingulum (hippocampus)_L', 'MD_Cingulum (hippocampus)_R', 'AD_Cingulum (hippocampus)_L', 'AD_Cingulum (hippocampus)_R', 'RD_Cingulum (hippocampus)_L', 'RD_Cingulum (hippocampus)_R']

}

# here is the logic, we iterate over each tract and for each tract, we iterate over the FA and MD columns.
# then we perform a pearson correlation between the logmratio (for each emotion) and the FA/MD values for each tract.
# in the occasion of a significant correlation, we plot the scatter plot with a line of best fit, reporting both the correlation coefficient and the p-value.
# we will do this for each emotion.



# create a dictionary to store the results
results = {}

# iterate over each tract
for tract, columns in tractdict.items():
    # create a dictionary to store the results for each tract
    results[tract] = {}
    # iterate over each FA and MD column
    for column in columns:
        # create a dictionary to store the results for each column
        results[tract][column] = {}
        # iterate over each emotion
        for emotion in logmratio_dti['Emotion'].unique():
            # create a dictionary to store the results for each emotion
            results[tract][column][emotion] = {}
            # get the logmratio and FA/MD values for the current emotion
            emotion_df = logmratio_dti[logmratio_dti['Emotion'] == emotion]
            logmratio = emotion_df['logMRatio']
            fa_md = emotion_df[column]
            # perform the pearson correlation
            correlation, pvalue = stats.pearsonr(logmratio, fa_md)
            # store the results
            results[tract][column][emotion]['correlation'] = correlation
            results[tract][column][emotion]['pvalue'] = pvalue
            # if the pvalue is significant, plot the scatter plot
            if pvalue < 0.05:
                fig = px.scatter(emotion_df, x='logMRatio', y=column, title=f'{column} vs logMRatio for {emotion}', trendline='ols'
                                , trendline_color_override='red', hover_data=['Name', 'ID', 'Emotion', column])
                fig.update_layout(yaxis_title=column, xaxis_title='logMRatio', width=1200, height=600)
                # add the correlation coefficient and pvalue to the plot
                fig.add_annotation(text=f"Correlation: {correlation:.2f}, p-value: {pvalue:.2f}", x=0.9, 
                                y = 1.1, 
                                    showarrow=False, 
                                    font=dict(
                                        family="Courier New, monospace",
                                        size=16,
                                        color="#ffffff"
                                    ), xref="paper", yref="paper", bgcolor="#1f77b4", bordercolor="#1f77b4", borderwidth=2, borderpad=4, opacity=0.8)
                fig.show()
            
